In [45]:
import pandas as pd
import numpy as np
import string
import re
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

df = pd.read_csv("temizlenmemis_tweets.csv")
df = df.drop(["Unnamed: 0"], axis=1)
df.head()

,username,tweet
0,acunilicali,İşte #Survivor2023’ün Gönüllüleri.. Tüm yarışm...
1,acunilicali,İşte #Survivor2023’ün Influencerları.. Tüm yar...
2,acunilicali,İşte #Survivor2023’ün Ünlüleri.. Tüm yarışmacı...
3,acunilicali,#OSesTürkiye Yılbaşı Özel programı bu akşam sa...
4,acunilicali,#SıfırıncıGün ilk bölümüyle birazdan #TV8’de b...


In [46]:
# veri içerisindeki bazı anlamsız kelimeler
stop_words = sorted(['škoda','ɐpuıʞɐʎ','ɐɯnɔ','ɹıpɹɐʌ','ɹıq','ɹǝƃuɐɹʇs','ɹǝɥ','ˇωˇ','έλληνα','έχω','ήρθες','αδερφέ','αμφιβολία','αυτή','γνωρίζω','δεν','δυο','είμαι','είναι','εμάς','ενθουσιάζουν','ζήσουμε','θα','κάνουν','και','καλώς','καμία','κοινά','μαζί','μας','με','μεγάλα','μιας','μου','οι','οικογένειά','ομάδα','ομάδες','παίζουν','παιχνίδια','παράδειγμα','πιστεύεις','πολλά','πολύ','που','πράγματα','πρόθυμα','πόλη','σε','στην','τα','την','το','των','υπέροχα','φιλίας','χαρούμενο','χαρούμενος','χωρών','όμορφης','όνειρα','ότι','ⁿᵉᵈᵉⁿ','くへ','ノspoiler','ヽつ','내일','단독공개','오후','𝓐𝓜𝓟𝓘','𝓑𝓲𝔃','𝓨𝓞𝓝','𝓲𝔃','𝓸𝓵𝓪𝓬𝓪𝓰'])

In [48]:
def clean(row):
    satirlar = row["tweet"].replace("\n", " ")
    satirlar = re.sub("[0-9]+", "", satirlar) # veri içerisindeki sayılar çıkarılıyor
    satirlar = [t for t in satirlar.split()if t not in stop_words] # veri içerisindeki anlamsız kelimeler çıkarılıyor
    return " ".join(satirlar).lower().translate(str.maketrans("", "", string.punctuation))

df["clean"] = df.apply(lambda row: clean(row), axis = 1)
df

,username,tweet,clean
0,acunilicali,İşte #Survivor2023’ün Gönüllüleri.. Tüm yarışm...,i̇şte survivor’ün gönüllüleri tüm yarışmacılar...
1,acunilicali,İşte #Survivor2023’ün Influencerları.. Tüm yar...,i̇şte survivor’ün influencerları tüm yarışmacı...
2,acunilicali,İşte #Survivor2023’ün Ünlüleri.. Tüm yarışmacı...,i̇şte survivor’ün ünlüleri tüm yarışmacılarımı...
3,acunilicali,#OSesTürkiye Yılbaşı Özel programı bu akşam sa...,osestürkiye yılbaşı özel programı bu akşam saa...
4,acunilicali,#SıfırıncıGün ilk bölümüyle birazdan #TV8’de b...,sıfırıncıgün ilk bölümüyle birazdan tv’de başl...
...,...,...,...
11460,netflixturkiye,"@mst553455 Disenchantment’ı aç, Elfo keyfini d...",mst disenchantment’ı aç elfo keyfini düzeltece...
11461,netflixturkiye,@AlkisResit The Midnight Sky izlendi mi? 👀,alkisresit the midnight sky izlendi mi 👀
11462,netflixturkiye,@hsyn11ozcn Kesinlikle Bodyguard,hsynozcn kesinlikle bodyguard
11463,netflixturkiye,Bugün ne izleyeyim yazıp cevap alamayanlar bu ...,bugün ne izleyeyim yazıp cevap alamayanlar bu ...


In [49]:
users = ["acunilicali", "RasimOzan_K", "webtekno", "netflixturkiye"]

In [50]:
def etiketle(row):
    if row["username"] == users[0]:
        return 0 
    elif row["username"] == users[1]:
        return 1
    elif row["username"] == users[2]:
        return 2
    return 3

df["labels"] = df.apply(lambda row:  etiketle(row), axis=1)
df

,username,tweet,clean,labels
0,acunilicali,İşte #Survivor2023’ün Gönüllüleri.. Tüm yarışm...,i̇şte survivor’ün gönüllüleri tüm yarışmacılar...,0
1,acunilicali,İşte #Survivor2023’ün Influencerları.. Tüm yar...,i̇şte survivor’ün influencerları tüm yarışmacı...,0
2,acunilicali,İşte #Survivor2023’ün Ünlüleri.. Tüm yarışmacı...,i̇şte survivor’ün ünlüleri tüm yarışmacılarımı...,0
3,acunilicali,#OSesTürkiye Yılbaşı Özel programı bu akşam sa...,osestürkiye yılbaşı özel programı bu akşam saa...,0
4,acunilicali,#SıfırıncıGün ilk bölümüyle birazdan #TV8’de b...,sıfırıncıgün ilk bölümüyle birazdan tv’de başl...,0
...,...,...,...,...
11460,netflixturkiye,"@mst553455 Disenchantment’ı aç, Elfo keyfini d...",mst disenchantment’ı aç elfo keyfini düzeltece...,3
11461,netflixturkiye,@AlkisResit The Midnight Sky izlendi mi? 👀,alkisresit the midnight sky izlendi mi 👀,3
11462,netflixturkiye,@hsyn11ozcn Kesinlikle Bodyguard,hsynozcn kesinlikle bodyguard,3
11463,netflixturkiye,Bugün ne izleyeyim yazıp cevap alamayanlar bu ...,bugün ne izleyeyim yazıp cevap alamayanlar bu ...,3


In [51]:
X = df.clean.to_numpy()
y = df.labels.to_numpy()

In [52]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [53]:
vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [54]:
model_NB = MultinomialNB()
model_NB.fit(X_train, y_train)

MultinomialNB()

In [56]:
print("NB train accuracy:", model_NB.score(X_train, y_train))
print("NB test accuracy:", model_NB.score(X_test, y_test))

predictions_train = model_NB.predict(X_train)

predictions_test = model_NB.predict(X_test)

NB train accuracy: 0.9604230266027038
NB test accuracy: 0.8656781508940253


In [57]:
model_DT = DecisionTreeClassifier()
model_DT.fit(X_train, y_train)

DecisionTreeClassifier()

In [58]:
print("DT train accuracy:", model_DT.score(X_train, y_train))
print("DT test accuracy:", model_DT.score(X_test, y_test))

predictions_train = model_DT.predict(X_train)

predictions_test = model_DT.predict(X_test)

DT train accuracy: 0.9997819450501526
DT test accuracy: 0.688181421718273


In [59]:
tweet = "önerdiğim diziyi izledin mi diyorum bana nau nau diyor"
tweet_vec = vectorizer.transform([tweet])
# acunilicali: 0
# rok: 1
# webtekno: 2
# netflix: 3
print("NB:", model_NB.predict(tweet_vec))
#print("DT:", model_DT.predict(tweet_vec))

result = model_NB.predict_proba(tweet_vec)
print("acun ılıcalı: %{}\nRasim Ozan Kütahyalı: %{}\nWebtekno: %{}\nNetflix: %{}".format(round(100*result[0][0],2),round(100*result[0][1],2),round(100*result[0][2],2),round(100*result[0][3],2)))
#print("DT Proba:", model_DT.predict_proba(tweet_vec))

NB: [3]
acun ılıcalı: %0.56
Rasim Ozan Kütahyalı: %6.95
Webtekno: %4.92
Netflix: %87.57
